In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import sklearn as sklearn
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.DataFrame()
df = pd.read_csv('C:/Users/Omotola/Desktop/MMA 869/train.csv', header='infer')

In [4]:
df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [5]:
df['EJ_A'] = np.where(df['EJ'] == 'A',1,0)
df['EJ_B'] = np.where(df['EJ'] == 'B',1,0)
df = df.drop(['EJ'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 59 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      617 non-null    object 
 1   AB      617 non-null    float64
 2   AF      617 non-null    float64
 3   AH      617 non-null    float64
 4   AM      617 non-null    float64
 5   AR      617 non-null    float64
 6   AX      617 non-null    float64
 7   AY      617 non-null    float64
 8   AZ      617 non-null    float64
 9   BC      617 non-null    float64
 10  BD      617 non-null    float64
 11  BN      617 non-null    float64
 12  BP      617 non-null    float64
 13  BQ      557 non-null    float64
 14  BR      617 non-null    float64
 15  BZ      617 non-null    float64
 16  CB      615 non-null    float64
 17  CC      614 non-null    float64
 18  CD      617 non-null    float64
 19  CF      617 non-null    float64
 20  CH      617 non-null    float64
 21  CL      617 non-null    float64
 22  CR

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop(['Class'],axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

type(X_train)
type(y_train)

pandas.core.series.Series

In [7]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="mean")

X_train.set_index('Id')
X_test.set_index('Id')
X_train = X_train.drop('Id',axis=1)
X_test = X_test.drop('Id',axis=1)

for x in X_train.columns:
    imputer = imputer.fit(X_train[[x]])
    X_train[x + "_Imputed"] = imputer.transform(X_train[[x]])
    X_train = X_train.drop([x],axis=1)

    X_test[x + "_Imputed"] = imputer.transform(X_test[[x]])
    X_test = X_test.drop([x],axis=1)

In [8]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124 entries, 49 to 181
Data columns (total 57 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   AB_Imputed    124 non-null    float64
 1   AF_Imputed    124 non-null    float64
 2   AH_Imputed    124 non-null    float64
 3   AM_Imputed    124 non-null    float64
 4   AR_Imputed    124 non-null    float64
 5   AX_Imputed    124 non-null    float64
 6   AY_Imputed    124 non-null    float64
 7   AZ_Imputed    124 non-null    float64
 8   BC_Imputed    124 non-null    float64
 9   BD _Imputed   124 non-null    float64
 10  BN_Imputed    124 non-null    float64
 11  BP_Imputed    124 non-null    float64
 12  BQ_Imputed    124 non-null    float64
 13  BR_Imputed    124 non-null    float64
 14  BZ_Imputed    124 non-null    float64
 15  CB_Imputed    124 non-null    float64
 16  CC_Imputed    124 non-null    float64
 17  CD _Imputed   124 non-null    float64
 18  CF_Imputed    124 non-null   

In [9]:
# Model 1: Decision Tree

In [10]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=42, criterion="entropy",
                             min_samples_split=10, min_samples_leaf=10, max_depth=3, max_leaf_nodes=5)
clf.fit(X_train, y_train)

y_pred_dt = clf.predict(X_test)

In [11]:
class_names = [str(x) for x in clf.classes_]

In [12]:
print(clf.tree_.node_count)
print(clf.tree_.impurity)
print(clf.tree_.children_left)
print(clf.tree_.threshold)

9
[0.66319684 0.45298117 0.954434   0.14144054 0.8567986  0.79732651
 0.55862937 0.         0.43055187]
[ 1  3 -1  7  5 -1 -1 -1 -1]
[ 2.28980398  0.49780449 -2.          1.47189504 29.32503986 -2.
 -2.         -2.         -2.        ]


In [13]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_dt)

array([[90, 11],
       [ 5, 18]], dtype=int64)

In [14]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_dt, target_names=class_names))

              precision    recall  f1-score   support

           0       0.95      0.89      0.92       101
           1       0.62      0.78      0.69        23

    accuracy                           0.87       124
   macro avg       0.78      0.84      0.81       124
weighted avg       0.89      0.87      0.88       124



In [15]:
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, log_loss

print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred_dt)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred_dt)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred_dt)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred_dt)))

Accuracy = 0.87
Kappa = 0.61
F1 Score = 0.69
Log Loss = 4.46


In [16]:
# Model 2: Random Forest (Bagging)

In [17]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    random_state=0)

clf_rf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [18]:
y_pred_dt = clf_rf.predict(X_test)

In [19]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_dt)

array([[101,   0],
       [  7,  16]], dtype=int64)

In [20]:
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, log_loss

print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred_dt)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred_dt)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred_dt)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred_dt)))

Accuracy = 0.94
Kappa = 0.79
F1 Score = 0.82
Log Loss = 1.95


In [21]:
import time

In [22]:
# Model 3: LightGBM

In [23]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate

# Split the dataframe into X (features) and y (target, aka, label).
X = X_train
y = y_train

params = {
      "cat_l2": 10,
      "cat_smooth": 10,
      "colsample_bytree": 0.8,
      "feature_fraction_bynode":  0.8,
      "learning_rate": 0.03,
      "max_depth": 6,
      "min_child_samples": 63,
      "min_data_per_group": 100,
      "n_estimators": 200,
      "num_leaves": 63,
      "path_smooth": 0,
      "reg_alpha":  0.05,
      "reg_lambda": 0.05,
      "subsample_freq": 1,
      "subsample": 0.8,
      "max_bin": 127,
      "extra_trees": False,
      "is_unbalance": True,
      "boosting_type": 'gbdt',
      "n_jobs": 1,
      "verbosity": -1,
      "seed": 77,
}
        
fit_params= {
    'feature_name': "auto",
}

start = time.time()
estimator = LGBMClassifier(**params)

# The more CV, the better our estimate of the score
inner_cv_scores = cross_validate(estimator, X, y, 
                                 fit_params=fit_params, 
                                 cv=15, 
                                 scoring="roc_auc", 
                                 n_jobs=5, 
                                 verbose=0, 
                                 return_train_score=True)
cv_scores=inner_cv_scores['test_score'].tolist()
duration = time.time() - start

print("CV Scores:")
print(["{:0.4f}".format(cv_score) for cv_score in cv_scores])
print("CV Score mean: {:.4f} ".format(np.mean(cv_scores)))
print("CV Score range: {:0.4f} -- {:0.4f}".format(np.mean(cv_scores) - np.std(cv_scores), np.mean(cv_scores) + np.std(cv_scores)))
print("Fit times: {}".format(duration))
print("Total duration: {}".format(duration))

CV Scores:
['1.0000', '0.9877', '0.9321', '0.9691', '1.0000', '0.9506', '0.9321', '0.9568', '0.8333', '0.9815', '0.9214', '1.0000', '0.9286', '1.0000', '0.8667']
CV Score mean: 0.9507 
CV Score range: 0.9022 -- 0.9992
Fit times: 21.089874029159546
Total duration: 21.089874029159546


In [24]:
# Of course, at this point, once you've found the best parameters etc., you would retrain the model on the full dataset:
estimator = LGBMClassifier(**params)
estimator = estimator.fit(X, y, **fit_params)

In [25]:
y_pred = estimator.predict(X_test)

In [26]:
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred)))

Accuracy = 0.94
Kappa = 0.82
F1 Score = 0.86
Log Loss = 1.95


In [27]:
# Model 4: XGBClassifier

In [29]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

params = {
      "learning_rate": 0.03,
      "max_depth": 6,
      "min_child_samples": 63,
      "min_data_per_group": 100,
      "n_estimators": 200,
      "num_leaves": 63,
      "reg_alpha":  0.05,
      "reg_lambda": 0.05,
      "subsample": 0.8,
      "boosting_type": 'gbtree',
      "n_jobs": 1,
      "verbosity": 0,
      "seed": 77,
}

pipe = Pipeline(steps=[
   #('encoder', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1, dtype=np.int32)),
   ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False, dtype=np.int32)),
   ('clf', XGBClassifier(**params)),                    
])

start = time.time()
pipe.fit(X, y)

# The more CV, the better our estimate of the score
inner_cv_scores = cross_validate(pipe, X, y, 
                                 cv=5, 
                                 scoring="roc_auc", 
                                 n_jobs=15, 
                                 verbose=0, 
                                 return_train_score=True)
cv_scores=inner_cv_scores['test_score'].tolist()
fit_times=inner_cv_scores['fit_time'].tolist()
duration = time.time() - start

print("CV Scores:")
print(["{:0.4f}".format(cv_score) for cv_score in cv_scores])
print("CV Score mean: {:.4f} ".format(np.mean(cv_scores)))
print("CV Score range: {:0.4f} -- {:0.4f}".format(np.mean(cv_scores) - np.std(cv_scores), np.mean(cv_scores) + np.std(cv_scores)))
print("Total duration: {}".format(duration))

CV Scores:
['0.7855', '0.7841', '0.7138', '0.7800', '0.8468']
CV Score mean: 0.7820 
CV Score range: 0.7399 -- 0.8241
Total duration: 166.58310461044312


In [30]:
estimator = XGBClassifier(**params)
estimator = estimator.fit(X, y)

In [31]:
y_pred = estimator.predict(X_test)

In [32]:
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred)))

Accuracy = 0.96
Kappa = 0.85
F1 Score = 0.88
Log Loss = 1.39


In [ ]:
# Model 5: XGBoost Tuned by Optuna

In [ ]:
pip install optuna

In [33]:
import optuna

In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

    
def objective_dt(trial, X, y):

  # Now, define all the hyperparams we want to vary, and what values they are allowed
  # to take.
  #
  # Each trial, optuna will automatically choose values for each hyperparam.
  hyper_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.005,1,step=0.005),
        'min_split_loss': trial.suggest_float('min_split_loss',0.1,10,step=0.1),
        'max_depth':trial.suggest_int('max_depth',1,12,step=1),
        'max_leaves':trial.suggest_int('max_leaves',0,1000,step=1),
        'min_child_weight':trial.suggest_float('min_child_weight',0,1,step=0.05),
      
        "n_estimators": trial.suggest_int("n_estimators", 10, 500, step=10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.01, 0.10, step=0.01),

        'reg_lambda': trial.suggest_float('reg_lambda', 0.01,0.10,step=0.01),

        "random_state": 77,
  }
    

  # Use the hyperparams that optuna has chosen for this trial to create a DecisionTreeClassifier
  clf = XGBClassifier(**hyper_params)

  # Run CV to see how well these hyper_params do
  cv_scores = cross_val_score(clf, X, y, cv=10, scoring="f1_macro")
  score = np.mean(cv_scores)

  # Whatever we return here tells optuna how well these parameters did
  return score

In [35]:
study = optuna.create_study(direction="maximize")

[I 2023-07-19 06:13:22,477] A new study created in memory with name: no-name-a5085133-cf1e-40bd-84d8-85cbf43fc8fc


In [36]:
study.optimize(lambda trial: objective_dt(trial, X, y), n_trials=100,  gc_after_trial=True)

[I 2023-07-19 06:13:40,624] Trial 0 finished with value: 0.7589121682874567 and parameters: {'learning_rate': 0.485, 'min_split_loss': 6.4, 'max_depth': 8, 'max_leaves': 136, 'min_child_weight': 0.7000000000000001, 'n_estimators': 290, 'reg_alpha': 0.08, 'reg_lambda': 0.01}. Best is trial 0 with value: 0.7589121682874567.
[I 2023-07-19 06:13:47,109] Trial 1 finished with value: 0.6954406400342185 and parameters: {'learning_rate': 0.24000000000000002, 'min_split_loss': 9.9, 'max_depth': 2, 'max_leaves': 562, 'min_child_weight': 0.9, 'n_estimators': 330, 'reg_alpha': 0.08, 'reg_lambda': 0.03}. Best is trial 0 with value: 0.7589121682874567.
[I 2023-07-19 06:13:47,920] Trial 2 finished with value: 0.7680619529459387 and parameters: {'learning_rate': 0.215, 'min_split_loss': 7.1, 'max_depth': 4, 'max_leaves': 362, 'min_child_weight': 0.8500000000000001, 'n_estimators': 20, 'reg_alpha': 0.04, 'reg_lambda': 0.09}. Best is trial 2 with value: 0.7680619529459387.
[I 2023-07-19 06:13:57,830] Tr

[I 2023-07-19 06:20:50,992] Trial 25 finished with value: 0.7844686876590815 and parameters: {'learning_rate': 0.049999999999999996, 'min_split_loss': 2.1, 'max_depth': 11, 'max_leaves': 627, 'min_child_weight': 0.45, 'n_estimators': 460, 'reg_alpha': 0.08, 'reg_lambda': 0.08}. Best is trial 11 with value: 0.8189938297225752.
[I 2023-07-19 06:21:20,664] Trial 26 finished with value: 0.7916733501600636 and parameters: {'learning_rate': 0.185, 'min_split_loss': 3.9000000000000004, 'max_depth': 9, 'max_leaves': 862, 'min_child_weight': 0.25, 'n_estimators': 480, 'reg_alpha': 0.02, 'reg_lambda': 0.09}. Best is trial 11 with value: 0.8189938297225752.
[I 2023-07-19 06:21:46,622] Trial 27 finished with value: 0.8017920814008399 and parameters: {'learning_rate': 0.08, 'min_split_loss': 1.0, 'max_depth': 10, 'max_leaves': 927, 'min_child_weight': 0.1, 'n_estimators': 400, 'reg_alpha': 0.09, 'reg_lambda': 0.05}. Best is trial 11 with value: 0.8189938297225752.
[I 2023-07-19 06:21:56,392] Trial 

[I 2023-07-19 06:24:10,923] Trial 50 finished with value: 0.8509813727468011 and parameters: {'learning_rate': 0.365, 'min_split_loss': 0.7000000000000001, 'max_depth': 2, 'max_leaves': 81, 'min_child_weight': 0.9, 'n_estimators': 180, 'reg_alpha': 0.02, 'reg_lambda': 0.01}. Best is trial 50 with value: 0.8509813727468011.
[I 2023-07-19 06:24:14,100] Trial 51 finished with value: 0.8298952499587559 and parameters: {'learning_rate': 0.395, 'min_split_loss': 0.6, 'max_depth': 2, 'max_leaves': 102, 'min_child_weight': 0.9, 'n_estimators': 180, 'reg_alpha': 0.02, 'reg_lambda': 0.01}. Best is trial 50 with value: 0.8509813727468011.
[I 2023-07-19 06:24:17,927] Trial 52 finished with value: 0.8662396727665242 and parameters: {'learning_rate': 0.36, 'min_split_loss': 0.1, 'max_depth': 3, 'max_leaves': 222, 'min_child_weight': 1.0, 'n_estimators': 240, 'reg_alpha': 0.03, 'reg_lambda': 0.02}. Best is trial 52 with value: 0.8662396727665242.
[I 2023-07-19 06:24:20,909] Trial 53 finished with val

[I 2023-07-19 06:25:36,474] Trial 76 finished with value: 0.83654647344548 and parameters: {'learning_rate': 0.84, 'min_split_loss': 1.7000000000000002, 'max_depth': 1, 'max_leaves': 63, 'min_child_weight': 0.8500000000000001, 'n_estimators': 140, 'reg_alpha': 0.01, 'reg_lambda': 0.01}. Best is trial 52 with value: 0.8662396727665242.
[I 2023-07-19 06:25:38,451] Trial 77 finished with value: 0.8479749498270334 and parameters: {'learning_rate': 0.925, 'min_split_loss': 1.2000000000000002, 'max_depth': 1, 'max_leaves': 166, 'min_child_weight': 0.8, 'n_estimators': 90, 'reg_alpha': 0.01, 'reg_lambda': 0.02}. Best is trial 52 with value: 0.8662396727665242.
[I 2023-07-19 06:25:40,817] Trial 78 finished with value: 0.8034898269733999 and parameters: {'learning_rate': 0.92, 'min_split_loss': 1.3000000000000003, 'max_depth': 2, 'max_leaves': 85, 'min_child_weight': 0.8, 'n_estimators': 90, 'reg_alpha': 0.01, 'reg_lambda': 0.02}. Best is trial 52 with value: 0.8662396727665242.
[I 2023-07-19 0

In [37]:
study.best_params

{'learning_rate': 0.36,
 'min_split_loss': 0.1,
 'max_depth': 3,
 'max_leaves': 222,
 'min_child_weight': 1.0,
 'n_estimators': 240,
 'reg_alpha': 0.03,
 'reg_lambda': 0.02}

In [38]:
study.best_value

0.8662396727665242

In [39]:
clf = XGBClassifier(**study.best_params)
clf.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.36, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=222,
              min_child_weight=1.0, min_split_loss=0.1, missing=nan,
              monotone_constraints=None, n_estimators=240, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [40]:
y_pred = clf.predict(X_test)

In [41]:
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred)))

Accuracy = 0.96
Kappa = 0.86
F1 Score = 0.88
Log Loss = 1.39


In [43]:
df = pd.read_csv( 'C:/Users/Omotola/Desktop/MMA 869/test.csv', header = 'infer' )
X_test = df

df['EJ_A'] = np.where(df['EJ'] == 'A',1,0)
df['EJ_B'] = np.where(df['EJ'] == 'B',1,0)
df = df.drop(['EJ'],axis=1)

In [44]:
X_test.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FR,FS,GB,GE,GF,GH,GI,GL,EJ_A,EJ_B
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [45]:
X_test_2 = df.drop(['Id'],axis=1)

In [46]:
X_test_2.columns.unique()

Index(['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
       'EE', 'EG', 'EH', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL', 'FR',
       'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'EJ_A', 'EJ_B'],
      dtype='object')

In [47]:
X_train.columns.unique()

Index(['AB_Imputed', 'AF_Imputed', 'AH_Imputed', 'AM_Imputed', 'AR_Imputed',
       'AX_Imputed', 'AY_Imputed', 'AZ_Imputed', 'BC_Imputed', 'BD _Imputed',
       'BN_Imputed', 'BP_Imputed', 'BQ_Imputed', 'BR_Imputed', 'BZ_Imputed',
       'CB_Imputed', 'CC_Imputed', 'CD _Imputed', 'CF_Imputed', 'CH_Imputed',
       'CL_Imputed', 'CR_Imputed', 'CS_Imputed', 'CU_Imputed', 'CW _Imputed',
       'DA_Imputed', 'DE_Imputed', 'DF_Imputed', 'DH_Imputed', 'DI_Imputed',
       'DL_Imputed', 'DN_Imputed', 'DU_Imputed', 'DV_Imputed', 'DY_Imputed',
       'EB_Imputed', 'EE_Imputed', 'EG_Imputed', 'EH_Imputed', 'EL_Imputed',
       'EP_Imputed', 'EU_Imputed', 'FC_Imputed', 'FD _Imputed', 'FE_Imputed',
       'FI_Imputed', 'FL_Imputed', 'FR_Imputed', 'FS_Imputed', 'GB_Imputed',
       'GE_Imputed', 'GF_Imputed', 'GH_Imputed', 'GI_Imputed', 'GL_Imputed',
       'EJ_A_Imputed', 'EJ_B_Imputed'],
      dtype='object')

In [48]:
X_test_2.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FR,FS,GB,GE,GF,GH,GI,GL,EJ_A,EJ_B
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [49]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="mean")


for x in X_test_2.columns:
    imputer = imputer.fit(X_test_2[[x]])
    X_test_2[x + "_Imputed"] = imputer.transform(X_test_2[[x]])
    X_test_2 = X_test_2.drop([x],axis=1)


In [50]:
y_pred = clf.predict(X_test_2)

In [51]:
y_pred

array([0, 0, 0, 0, 0])

In [52]:
X_test_2

,AB_Imputed,AF_Imputed,AH_Imputed,AM_Imputed,AR_Imputed,AX_Imputed,AY_Imputed,AZ_Imputed,BC_Imputed,BD _Imputed,...,FR_Imputed,FS_Imputed,GB_Imputed,GE_Imputed,GF_Imputed,GH_Imputed,GI_Imputed,GL_Imputed,EJ_A_Imputed,EJ_B_Imputed
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [54]:
submission = pd.DataFrame(X_test['Id'], columns=['Id'])
class_0 = []
class_1 = []

for x in y_pred:
    if x == 1:
        class_0.append(0)
        class_1.append(1)
    else:
        class_0.append(1)
        class_1.append(0)
        
submission['class_0'] = class_0
submission['class_1'] = class_1
submission.to_csv('C:/Users/Omotola/Desktop/MMA 869/submission.csv', index=False) 